In [7]:
import os
import csv 
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from langchain_together import Together
from langchain_community.document_loaders import CSVLoader

load_dotenv()

True

---

## Document Loading

In [2]:
file_path = "./Data/beauty_test_amazon.csv"
csv.field_size_limit(10**6)
loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
    },
)

In [3]:
data = loader.load()

In [4]:
len(data)

19162

---

## Text Splitting

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter  = RecursiveCharacterTextSplitter(
    chunk_size = 1500,chunk_overlap=100,add_start_index=True
)
all_splits = text_splitter.split_documents(data)

In [21]:
len(all_splits)

35769

---

## Embeddings

In [10]:
from langchain_together import TogetherEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

/mnt/Files/Sorbonne_Data_Analytics_SDA/Cours/Gen_AI_Dafnis-Krasniqi/Gift_Recommendation_ChatBot/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [13]:
text = all_splits[0]
text.page_content

'parent_asin: B07NGFDN6G\naverage_rating: 4.3\nmain_category: All Beauty\ntitle: Precision Plunger Bars for Cartridge Grips – 93mm – Bag of 10 Plungers\ncomment: Missing Review.......... It’s lost at Amazon Somewhere Where did my review go? I used your take a picture link and came back and my review was gone'

In [15]:
%%time
vector = embeddings.embed_query(all_splits[0].page_content)

CPU times: user 6.5 ms, sys: 775 μs, total: 7.27 ms
Wall time: 6.63 ms


[-0.05531195551156998,
 0.02398253045976162,
 -0.004908147733658552,
 0.046251773834228516,
 -0.053958017379045486,
 -0.024386055767536163,
 0.011282631196081638,
 0.024630649015307426,
 -0.03853785619139671,
 -0.014416935853660107,
 -0.0008630677475593984,
 -0.002616514451801777,
 -0.0012694557663053274,
 0.012978953309357166,
 -0.10460236668586731,
 -0.022581780329346657,
 -0.032895032316446304,
 -0.041898537427186966,
 -0.03291773051023483,
 -0.017513033002614975,
 -0.11115191876888275,
 0.05115780606865883,
 0.07591526955366135,
 0.037660930305719376,
 -0.026323532685637474,
 -0.0060433316975831985,
 -0.0635593980550766,
 0.024236151948571205,
 -0.018433338031172752,
 -0.07537619769573212,
 0.020151177421212196,
 0.044248636811971664,
 -0.055580008774995804,
 -0.009857946075499058,
 0.024429071694612503,
 -0.0678430125117302,
 0.005527735222131014,
 -0.05108940601348877,
 0.06739599257707596,
 -0.009822611697018147,
 0.0030173189006745815,
 -0.04593724012374878,
 0.0363161005079746

In [24]:
from langchain_chroma import Chroma
persist_directory = './chroma_vectorstore/'
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings,persist_directory=persist_directory)

In [11]:
### Load the vectorstore
from langchain_chroma import Chroma
persist_directory = './chroma_vectorstore/'
vectorstore = Chroma(persist_directory=persist_directory,embedding_function=embeddings)

---

## Retrieval and Generation

In [12]:
query = "I am looking for a gift for my 30 year old mom. She loves makeup and has a light skin. she loves the friends show"

In [13]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":6})
retrieved_docs=retriever.invoke(query)
len(retrieved_docs)

6

---

## RAG

In [14]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(base_url="https://api.together.xyz/v1",api_key=os.getenv("TOGETHER_API_KEY"),model="meta-llama/Llama-Vision-Free")

In [15]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

In [89]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    input_key="question",
    memory_key="history",
    #output_key="result",
    return_messages=True,
)

In [65]:
question = "I am looking for a gift for my 30 year old mom. She loves makeup and has a light skin. she loves the friends show"

In [96]:
from langchain.prompts import PromptTemplate

# Build prompt

template = """You are an intelligent gift recommendation assistant designed to help users find the perfect gifts based on their preferences, interests, and occasions. Your goal is to provide personalized gift suggestions by retrieving relevant information from a vast database of a retailer's products and combining it with natural language generation to create engaging and informative responses.

User Preferences: Begin by gathering information about the user's preferences. Ask questions related to the recipient’s age, gender, interests, hobbies, and any specific occasions (e.g., birthdays, anniversaries, holidays).

Product Retrieval: Utilize the retailer's catalog to retrieve a curated list of products that match the user's criteria. Ensure the recommendations cover various categories. Recommend preferably and when you have the choice high rated products. Always remember the products are from the retailer's catalog and are not given by the user.

Personalized Suggestions: Generate personalized gift suggestions with clear and concise descriptions of each item. Assume that the user may not be familiar with the products, so include relevant details like features, benefits, and why each item would make a great gift for the recipient.

Contextual Information: Provide context for each recommendation, such as the target audience (e.g., kids, teenagers, adults) and any trending attributes (e.g., popularity, user ratings) that may help the user make an informed decision.

Follow-up Questions: Encourage user interaction by asking follow-up questions to refine suggestions further. This may include inquiring about the recipient’s preferences for brands, styles, or colors.

Feedback Loop: Implement a feedback mechanism where users can rate suggestions and provide comments. Use this feedback to improve future recommendations.

Tone and Style: Maintain a friendly, engaging, and conversational tone throughout the interaction to make the experience enjoyable for the user. Use simple language that is easy to understand, avoiding jargon or technical terms.

Remember to keep the conversation flowing naturally, and be ready to adapt your responses based on user input. Your goal is to help users find a gift that will delight the recipient, making their shopping experience seamless and enjoyable!
Question: {question}
Context: {context}
history: {history}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)


# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT,"verbose":True,"memory":memory})

In [91]:
result = qa_chain({"query": question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an intelligent gift recommendation assistant designed to help users find the perfect gifts based on their preferences, interests, and occasions. Your goal is to provide personalized gift suggestions by retrieving relevant information from a vast database of a retailer's products and combining it with natural language generation to create engaging and informative responses.

User Preferences: Begin by gathering information about the user's preferences. Ask questions related to the recipient’s age, gender, interests, hobbies, and any specific occasions (e.g., birthdays, anniversaries, holidays).

Product Retrieval: Utilize the retailer's catalog to retrieve a curated list of products that match the user's criteria. Ensure the recommendations cover various categories. Always remember the products are from the retailer's catalog and are not given by the user.

Personalized Sugg

In [92]:
result

{'query': 'I am looking for a gift for my 30 year old mom. She loves makeup and has a light skin. she loves the friends show',
 'result': "It sounds like your mom would love some makeup goodies, especially since she's a fan of Friends! I've got some gift ideas that might just make her day.\n\nBased on your input, I've selected a few products from our retailer's catalog that cater to her interests and skin tone. Here are my top recommendations:\n\n1. **Makeup Revolution X Friends Eyeshadow Palette (Rachel)**: This palette is a great gift for any Friends fan, and it's perfect for everyday wear. It features 12 eyeshadows in a range of neutral shades, from light to dark. The palette is compact and travel-friendly, making it a great addition to any makeup bag. ($25)\n\t* Target audience: Adults, especially Friends fans\n\t* Trending attribute: Collaborations are always popular, and this one is a must-have for any Friends enthusiast\n2. **Lady De Exclusive Leopard Skin Makeup Kit, Dance Kit 

In [93]:
result = qa_chain({"query": "What do you remember from our conversation?"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an intelligent gift recommendation assistant designed to help users find the perfect gifts based on their preferences, interests, and occasions. Your goal is to provide personalized gift suggestions by retrieving relevant information from a vast database of a retailer's products and combining it with natural language generation to create engaging and informative responses.

User Preferences: Begin by gathering information about the user's preferences. Ask questions related to the recipient’s age, gender, interests, hobbies, and any specific occasions (e.g., birthdays, anniversaries, holidays).

Product Retrieval: Utilize the retailer's catalog to retrieve a curated list of products that match the user's criteria. Ensure the recommendations cover various categories. Always remember the products are from the retailer's catalog and are not given by the user.

Personalized Sugg

In [95]:
result

{'query': 'What do you remember from our conversation?',
 'result': "It looks like we've had a nice conversation about gift ideas for your 30-year-old mom. I remember that she loves makeup and has a light skin tone, and she's also a fan of the TV show Friends.\n\nI provided you with three gift recommendations that cater to her interests and skin tone:\n\n1. **Makeup Revolution X Friends Eyeshadow Palette (Rachel)**: This compact eyeshadow palette is perfect for everyday wear and features 12 neutral shades.\n2. **Lady De Exclusive Leopard Skin Makeup Kit, Dance Kit 2**: Although this kit is geared towards younger girls, the fun and playful design might appeal to your mom.\n3. **Theatrical Makeup Kits - Fair: Light PK-1**: This kit includes a range of makeup products to help your mom achieve a flawless, theatrical look.\n\nBefore finalizing these recommendations, I asked a few follow-up questions to ensure I get it just right. Unfortunately, I don't see any follow-up responses from you. 

---